In [ ]:
!pip install langchain==0.3.0 langchain-openai==0.2.0 langgraph==0.2.22

In [ ]:
# OpenAIのAPIキーを環境変数に設定（ハードコードは避けてください）
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY" # セキュリティのため実際は.envファイルなどが望ましい

In [ ]:
import operator
from typing import Annotated, Any
from langchain_core.pydantic_v1 import BaseModel, Field  # 状態モデル用
from langgraph.graph import StateGraph, END  # グラフ構築用
from langchain_openai import ChatOpenAI  # OpenAIのチャットモデル
from langchain_core.runnables import ConfigurableField  # LLM設定変更用
from langchain_core.prompts import ChatPromptTemplate  # プロンプトテンプレート
from langchain_core.output_parsers import StrOutputParser  # LLMの文字列出力を扱う

In [ ]:
# 回答ロールの定義（番号付き）
ROLES = {
	"1": {
		"name": "一般知識エキスパート",
		"description": "幅広い分野の一般的な質問に答える",
		"details": "幅広い分野の一般的な質問に対して、正確でわかりやすい回答を提供してください。"
	},
	"2": {
		"name": "生成AI製品エキスパート",
		"description": "生成AIや関連製品、技術に関する専門的な質問に答える",
		"details": "生成AIや関連製品、技術に関する専門的な質問に対して、最新の情報と深い洞察を提供してください。"
	},
	"3": {
		"name": "カウンセラー",
		"description": "個人的な悩みや心理的な問題に対してサポートを提供する",
		"details": "個人的な悩みや心理的な問題に対して、共感的で支援的な回答を提供し、可能であれば適切なアドバイスも行なってください。"
	}
}

In [ ]:
# LangGraphに渡す状態の定義
class State(BaseModel):
	query: str = Field(..., description="ユーザーからの質問")
	current_role: str = Field(
		default="", description="選定された回答ロール"
	)
	messages: Annotated[list[str], operator.add] = Field(
		default=[], description="回答履歴"
	)
	current_judge: bool = Field(
		default=False, description="品質チェックの結果"
	)
	judgment_reason: str = Field(
		default="", description="品質チェックの判定理由"
	)

In [ ]:
# OpenAIのチャットモデル（gpt-4o-mini）を初期化
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
llm = llm.configurable_fields(max_tokens=ConfigurableField(id='max_tokens'))

In [ ]:
# ノード①：ロールの選定
def selection_node(state: State) -> dict[str, Any]:
	query = state.query
	role_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])
	prompt = ChatPromptTemplate.from_template(
"""質問を分析し、最も適切な回答担当ロールを選択してください。

選択肢:
{role_options}

回答は選択肢の番号（1、2、または3）のみを返してください。

質問: {query}
""".strip()
	)

	# max_tokens=1 に設定して、1文字（番号）のみ返すよう制限
	chain = prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
	role_number = chain.invoke({"role_options": role_options, "query": query})

	selected_role = ROLES[role_number.strip()]["name"]
	return {"current_role": selected_role}

In [ ]:
# ノード②：ロールに応じた回答の生成
def answering_node(state: State) -> dict[str, Any]:
	query = state.query
	role = state.current_role
	role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])
	prompt = ChatPromptTemplate.from_template(
"""あなたは{role}として回答してください。以下の質問に対して、あなたの役割に基づいた適切な回答を提供してください。

役割の詳細:
{role_details}

質問: {query}

回答:""".strip()
	)
	chain = prompt | llm |StrOutputParser()
	answer = chain.invoke({"role": role, "role_details": role_details, "query": query})

	return {"messages": [answer]}

In [ ]:
# 回答品質の評価スキーマ
class Judgement(BaseModel):
	judge: bool = Field(default=False, description="判定結果")
	reason: str = Field(default="", description="判定理由")

In [ ]:
# ノード③：品質チェック（OKなら終了、NGなら再実行）
def check_node(state: State) -> dict[str, Any]:
	query = state.query
	answer = state.messages[-1]
	prompt = ChatPromptTemplate.from_template(
"""以下の回答の品質をチェックし、問題がある場合は'False'、問題がない場合は'True'を回答してください。
また、その判断理由も説明してください。

ユーザーからの質問: {query}
回答: {answer}
""".strip()
	)
	chain = prompt | llm.with_structured_output(Judgement)
	result: Judgement = chain.invoke({"query": query, "answer": answer})

	return {
		"current_judge": result.judge,
		"judgement_reason": result.reason
	}


In [ ]:
# LangGraphグラフの構築
workflow = StateGraph(State)

# 各ノードをグラフに追加
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

# 処理開始点を指定（最初はロール選定から）
workflow.set_entry_point("selection")

# 各ノード間の遷移（エッジ）を定義
workflow.add_edge("selection", "answering")
workflow.add_edge("answering", "check")

# 条件分岐：checkノードの結果によって終了 or やり直し
workflow.add_conditional_edges(
	"check",
	lambda state: state.current_judge, # Trueなら終了、Falseなら再実行
	{True: END, False: "selection"}
)

In [ ]:
# グラフの最終的なコンパイル（実行可能状態に）
compiled = workflow.compile()

# 初期状態としてユーザーの質問を与える
initial_state = State(query="生成AIについて教えてください")

# 実行（LangGraphが自動でループや遷移を管理）
result = compiled.invoke(initial_state)

# 最終出力（最終回答メッセージ）
print(result["messages"][-1])

In [ ]:
!apt-get install graphviz libgraphviz-dev pkg-config
!pip install pygraphviz

In [ ]:
from IPython.display import Image
Image(compiled.get_graph().draw_png())